# 패키지 import

In [1]:
import pandas as pd
import matplotlib as mtp
import numpy as np

# 시각화 패키지
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime, timedelta

# 데이터 수집 - 뉴스 기사 크롤링

In [2]:
# selenium 설치를 위한 작업
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# ChromeDriver 자동 설치 및 실행
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [3]:
# 데이터 크롤링을 위한 라이브러리 설치
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

webdriver_service = Service('/usr/bin/chromedriver')

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# ChromeDriver 자동 설치 및 실행
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

import requests as req
from bs4 import BeautifulSoup as bs
from datetime import datetime, timedelta

In [8]:
from datetime import datetime, timedelta
from tqdm import tqdm
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

# WebDriver 설정
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

try:
    # ChromeDriver 자동 설치 및 실행
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
except Exception as e:
    print(f"드라이버 오류 발생: {e}")
    exit()

# yg 엔터테이먼트 키워드를 포함하는 기사 검색 URL
url = 'https://search.naver.com/search.naver?sm=tab_hty.top&where=news&ssc=tab.news.all&query=%EC%9E%88%EC%A7%80&oquery=2PM&tqi=i20LddqVOsosshZrg20ssssss7s-131928&pd=3&photo=0&sort=0'

title_lists = []

start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 12, 26)

# 전체 날짜 범위 계산 (일 단위 진행률 표시용)
total_days = (end_date - start_date).days + 1

# tqdm을 이용한 진행 상황 표시
with tqdm(total=total_days, desc="진행 상황") as pbar:
    # 월 단위로 날짜 범위 생성
    current_date = start_date
    while current_date <= end_date:
        # 해당 월의 시작과 끝 날짜 설정
        month_start = current_date
        month_end = (current_date.replace(day=1) + timedelta(days=31)).replace(day=1) - timedelta(days=1)
        if month_end > end_date:
            month_end = end_date

        # 월 단위로 일자 생성하여 반복문 수행
        for single_date in (month_start + timedelta(days=x) for x in range((month_end - month_start).days + 1)):
            try:
                # 검색 URL 생성
                formatted_date = single_date.strftime('%Y%m%d')
                search_url = f"{url}&ds={formatted_date}&de={formatted_date}"

                # 페이지 이동
                driver.get(search_url)
                time.sleep(2)  # 페이지 로딩 시간 대기

                # 기사 제목 가져오기
                titles = driver.find_elements(By.CSS_SELECTOR, 'a.news_tit')
                title_list = [formatted_date]  # 날짜 추가
                for title in titles:
                    title_list.append(title.text)
                title_lists.append(title_list)

            except Exception as e:
                print(f"에러 발생 날짜: {formatted_date} - {e}")

            # tqdm 업데이트
            pbar.update(1)

        # 다음 월로 이동
        current_date = (current_date.replace(day=1) + timedelta(days=31)).replace(day=1)

# 웹 드라이버 종료
driver.quit()

# DataFrame 저장
df = pd.DataFrame(title_lists)
df.to_csv("news_title_jyp_있지.csv", index=False, encoding='utf-8-sig')
print("작업 완료 및 파일 저장 완료")



진행 상황: 100%|█████████████████████████████████████████████████████████████████████| 361/361 [16:23<00:00,  2.72s/it]


작업 완료 및 파일 저장 완료
